## Classifier model 
(since linear regression doesnt seem to work when we only look at the weather)

#### Load dataframe, sort to 1984-2023, change date column into date format

In [ ]:
#import both dataframe (fire and weather), sort to 1984-2023
import pandas as pd
weather = pd.read_csv(r'weather_pivoted.csv')
fire = pd.read_csv(r'fire data with 83 firestn.csv')


weather['date']= pd.to_datetime(weather['date'])
fire['ALARM_DATE'] = pd.to_datetime(fire['ALARM_DATE'])

weather = weather.loc[(weather['date'] >= '1984-01-01') & (weather['date'] <= '2023-12-31')]


#weather.set_index(['date'])
#fire.set_index(['ALARM_DATE'])



C:\Users\natal\AppData\Local\Temp\ipykernel_35960\737385975.py:7: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  fire['ALARM_DATE'] = pd.to_datetime(fire['ALARM_DATE'])


### Merge 2 dataframe and add a column to indicate fire occurence (1/0)

In [ ]:
#merge dataframes

weather_fire_merged = weather.merge(fire, left_on='date', right_on='ALARM_DATE', how='left')

#adding a column to indicate whether there is a fire on the date
weather_fire_merged['Fire_occur'] = weather_fire_merged['GIS_ACRES'].notna().astype(int)




In [28]:
weather_fire_merged= pd.read_csv(r'weather_fire_merged pivoted.csv')
a = weather_fire_merged.isnull().sum()
weather_fire_merged = weather_fire_merged.drop(columns=weather_fire_merged.columns[weather_fire_merged.isnull().sum() > 400000])

# Drop specified columns
weather_fire_class = weather_fire_merged.drop(columns=['GIS_ACRES', 'Shape__Area', 'Shape__Length'])



pd.merge on column is too slow, try making id and date index now

### Classification model: Random forest classifier

** have not done it so this is just a template: 260225**

In [29]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Ensure ALARM_DATE is in datetime format
weather_fire_merged['ALARM_DATE'] = pd.to_datetime(weather_fire_merged['ALARM_DATE'])

# 🔹 Feature Engineering for Time Series Classification
weather_fire_merged['month'] = weather_fire_merged['ALARM_DATE'].dt.month  # Capture seasonality
weather_fire_merged['year'] = weather_fire_merged['ALARM_DATE'].dt.year    # Capture trend

# 🔹 Define Features and Target
X = weather_fire_merged.loc[:, 'PRCP':'TOBS']  # Select all columns from PRCP to TOBS
X['month'] = weather_fire_merged['month']      # Add month as a feature
y = weather_fire_merged['Fire_occur']          # Target variable (binary)

# 🔹 Ensure `y` is numeric (in case it's not already)
y = y.astype(int)  # 1 = Fire occurred, 0 = No Fire

# 🔹 Split the data by location to ensure trends are captured per location
locations = weather_fire_merged['weatherstn'].unique()

X_train_list = []
y_train_list = []
X_test_list = []
y_test_list = []

# 🔹 Split data by location (NO RANDOM SPLIT)
for loc in locations:
    loc_data = weather_fire_merged[weather_fire_merged['weatherstn'] == loc]
    
    # 🔹 Use time-based train-test split
    train_loc = loc_data[loc_data['year'] < 2015]  # Train on years before 2015
    test_loc = loc_data[loc_data['year'] >= 2015]  # Test on 2015 and beyond
    
    X_train_list.append(train_loc.loc[:, 'PRCP':'TOBS'].assign(month=train_loc['month']))
    y_train_list.append(train_loc['Fire_occur'])
    
    X_test_list.append(test_loc.loc[:, 'PRCP':'TOBS'].assign(month=test_loc['month']))
    y_test_list.append(test_loc['Fire_occur'])

# 🔹 Concatenate data for training and testing (location-specific)
X_train = pd.concat(X_train_list)
y_train = pd.concat(y_train_list)
X_test = pd.concat(X_test_list)
y_test = pd.concat(y_test_list)

# 🔹 Define and Train the RandomForestClassifier model
model = RandomForestClassifier(random_state=42, n_estimators=100, class_weight='balanced')
model.fit(X_train, y_train)

# 🔹 Make Predictions
y_pred = model.predict(X_test)

# 🔹 Evaluate the Classification Model
print("🔥 Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))
print("\n🧩 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


🔥 Accuracy: 1.0

📊 Classification Report:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00     97333

    accuracy                           1.00     97333
   macro avg       1.00      1.00      1.00     97333
weighted avg       1.00      1.00      1.00     97333


🧩 Confusion Matrix:
 [[97333]]


c:\Users\natal\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [31]:
print(weather_fire_merged['Fire_occur'].value_counts())



Fire_occur
0    270065
1    225800
Name: count, dtype: int64


In [32]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Ensure ALARM_DATE is in datetime format
weather_fire_merged['ALARM_DATE'] = pd.to_datetime(weather_fire_merged['ALARM_DATE'])

# Feature Engineering for Time Series Classification
weather_fire_merged['month'] = weather_fire_merged['ALARM_DATE'].dt.month  # Capture seasonality
weather_fire_merged['year'] = weather_fire_merged['ALARM_DATE'].dt.year    # Capture trend

# Define Features and Target
X = weather_fire_merged.loc[:, 'PRCP':'TOBS']  # Select all columns from PRCP to TOBS
X['month'] = weather_fire_merged['month']      # Add month as a feature
y = weather_fire_merged['Fire_occur']          # Target variable (binary)

# Ensure `y` is numeric (in case it's not already)
y = y.astype(int)  # 1 = Fire occurred, 0 = No Fire

# Split the data into train and test (stratified by 'Fire_occur' to keep class balance)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Check the class distribution in the training and testing sets
print("Train class distribution:\n", y_train.value_counts())
print("Test class distribution:\n", y_test.value_counts())

# Define and Train the RandomForestClassifier model
model = RandomForestClassifier(random_state=42, n_estimators=100, class_weight='balanced')
model.fit(X_train, y_train)

# Make Predictions
y_pred = model.predict(X_test)

# Evaluate the Classification Model
print("🔥 Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))
print("\n🧩 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Train class distribution:
 Fire_occur
0    216052
1    180640
Name: count, dtype: int64
Test class distribution:
 Fire_occur
0    54013
1    45160
Name: count, dtype: int64
🔥 Accuracy: 1.0

📊 Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     54013
           1       1.00      1.00      1.00     45160

    accuracy                           1.00     99173
   macro avg       1.00      1.00      1.00     99173
weighted avg       1.00      1.00      1.00     99173


🧩 Confusion Matrix:
 [[54013     0]
 [    0 45160]]


In [33]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print("Cross-validated accuracy scores:", cv_scores)
print("Average cross-validated accuracy:", cv_scores.mean())


Cross-validated accuracy scores: [1. 1. 1. 1. 1.]
Average cross-validated accuracy: 1.0


### now that looks too good to be true: we look at the feature importance to see which one is the dominating factor

### Month(seasonality) contribute the most: try dropping month and see if the model can still perform well

Seasonal encoding: You can use sinusoidal encoding (sine and cosine transformations) for month and day features to capture periodicity without allowing the raw values to dominate. This allows the model to interpret months as a continuous cycle.

In [40]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Ensure ALARM_DATE is in datetime format
weather_fire_merged['ALARM_DATE'] = pd.to_datetime(weather_fire_merged['ALARM_DATE'])

# Feature Engineering for Time Series Classification
#weather_fire_merged['month'] = weather_fire_merged['ALARM_DATE'].dt.month  # Capture seasonality
#weather_fire_merged['year'] = weather_fire_merged['ALARM_DATE'].dt.year    # Capture trend
weather_fire_merged['month_sin'] = np.sin(2 * np.pi * weather_fire_merged['month'] / 12)
weather_fire_merged['month_cos'] = np.cos(2 * np.pi * weather_fire_merged['month'] / 12)

# Define Features and Target
X = weather_fire_merged.loc[:, 'PRCP':'TOBS']  # Select all columns from PRCP to TOBS
#X = X.assign(month_sin=weather_fire_merged['month_sin'], month_cos=weather_fire_merged['month_cos'])
y = weather_fire_merged['Fire_occur']          # Target variable (binary)

# Ensure `y` is numeric (in case it's not already)
y = y.astype(int)  # 1 = Fire occurred, 0 = No Fire

# Split the data into train and test (stratified by 'Fire_occur' to keep class balance)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Check the class distribution in the training and testing sets
print("Train class distribution:\n", y_train.value_counts())
print("Test class distribution:\n", y_test.value_counts())

# Define and Train the RandomForestClassifier model
model = RandomForestClassifier(random_state=42, n_estimators=100, class_weight='balanced')




# Train the model with the modified data (without the 'month' column)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate accuracy without the 'month' feature
print("Accuracy without month feature:", accuracy_score(y_test, y_pred))
# Evaluate the Classification Model
print("🔥 Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))
print("\n🧩 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Train class distribution:
 Fire_occur
0    216052
1    180640
Name: count, dtype: int64
Test class distribution:
 Fire_occur
0    54013
1    45160
Name: count, dtype: int64
Accuracy without month feature: 0.7683744567573836
🔥 Accuracy: 0.7683744567573836

📊 Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.76      0.78     54013
           1       0.73      0.78      0.75     45160

    accuracy                           0.77     99173
   macro avg       0.77      0.77      0.77     99173
weighted avg       0.77      0.77      0.77     99173


🧩 Confusion Matrix:
 [[41131 12882]
 [10089 35071]]


In [41]:
# Test set evaluation
y_pred = model.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))


Test Accuracy: 0.7683744567573836


In [42]:
feature_importances = model.feature_importances_
features = X.columns
sorted_idx = np.argsort(feature_importances)

print("Feature importances (sorted):")
for idx in sorted_idx:
    print(f"{features[idx]}: {feature_importances[idx]}")


Feature importances (sorted):
SNOW: 0.008416253274974257
SNWD: 0.020757238418943492
PRCP: 0.045731719645994866
TOBS: 0.13087257514941222
TAVG: 0.16771873591095848
TMIN: 0.2649587467413492
TMAX: 0.36154473085836747
